In [1]:
import pandas as pd

In [2]:
mimic_data  = pd.read_csv("data.csv")
mimic_data = mimic_data
print(list(mimic_data))

['Unnamed: 0', 'ROW_ID_x', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT', 'ROW_ID_y', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA']


CHART_TIME = Time at which the observation was noted down by the nurse
CHART_DATE = CHARTDATE is equivalent to CHARTTIME, except it does not contain any information on the time (all hour, minute, and seconds are 0 for these measurements).
STORE_TIME = The STORETIME provides the exact time that this validation occurred
CATEGORY = CATEGORY is a integer which identifies the category of the CPT code.
DESCRIPTION = Refers to the type
CGID = CGID is the identifier for the caregiver who validated the given measurement
ISERROR = A ‘1’ in the ISERROR column indicates that a physician has identified this note as an error. (I think this is associated with the CG)
ADMITTIME = Time at which the patient was admitted
DISCHTIME = Time at which the patient was discharged
DEATHTIME = Time at which the patient died.
ADMISSION_TYPE = ADMISSION_TYPE describes the type of the admission: ‘ELECTIVE’, ‘URGENT’, ‘NEWBORN’ or ‘EMERGENCY’. Emergency/urgent indicate unplanned medical care, and are often collapsed into a single category in studies. 
ADMISSION_LOCATION = ADMISSION_LOCATION provides information about the previous location of the patient prior to arriving at the hospital.
DISCHARGE_LOCATION = DISCHARGE_LOCATION provides information about the discharge location ofthe patient.
INSURANCE = Tells if the patient has any kind of insurance
LANGUAGE = Tells what the language of the patient is
RELIGION = Tells the religion of the patient
MARITAL_STATUS = Tells the marita status of the patient 
ETHNICITY = Tells the ethnicity
EDREGTIME = Time that the patient was registered from emergency department
EDOUTTIME = Time patient was discharged from the emergency department
DIAGNOSIS = Diagnosis of the patient
HOSPITAL_EXPIRE_FLAG = Probably means if the patient expired in the hospital or not
HAS_CHARTEVENTS_DATA = 
    
Identifiers which specify the patient: SUBJECT_ID is unique to a patient and HADM_ID is unique to a patient hospital stay.

In [3]:
pat_set = set()
data = mimic_data.as_matrix()
final_structure = []
for entry in data:
    subject_id = entry[2]
    stay_id = entry[3]
    store_time = entry[6]
    chart_time = entry[5]
    category = entry[7]
    admit_time = entry[13]
    disch_time = entry[14]
    death_time = entry[15]
    admission_type = entry[16]
    admission_location = entry[17]
    discharge_location = entry[18]
    insurance = entry[19]
    language = entry[20]
    religion = entry[21]
    marital_status = entry[22]
    ethnicity = entry[23]
    edreg_time = entry[24]
    edout_time = entry[23]
    diagnosis = entry[26]
    
    if subject_id not in pat_set:
        # then we need to construct a new tuple with the entries
        pat_set.add(subject_id)
        outer_struct = {'pat_id':subject_id, 'hospital':[], 'death_time': death_time, 'insurance': insurance,
                       'language': language, 'religion': religion, 'marital_status': marital_status, 
                       'ethnicity': ethnicity}
        inner_struct = {'stay_id':stay_id, 'diagnosis':diagnosis, 'chart_time':[chart_time],
                       'admit_time':admit_time, 'disch_time': disch_time, 'admission_type': admission_type,
                       'admission_location': admission_location, 'discharge_location': discharge_location,
                        'edreg_time': edreg_time, 'edout_time': edout_time
                       } 
        outer_struct['hospital'].append(inner_struct)
        final_structure.append(outer_struct)
    else:
        for items in final_structure:
            if subject_id == items['pat_id']:
                items['hospital'][0]['chart_time'].append(chart_time)
        # in this case it means there exists multiple cases of diagnosis for the same pat_id, hamd_id.
        
print(final_structure[0])

{'pat_id': 22532, 'hospital': [{'stay_id': 167853.0, 'diagnosis': 'RT LOWER LOBE PNEUMONIA', 'chart_time': [nan, nan, nan, nan, nan, nan, nan, nan, '2151-07-16 05:01:00', '2151-08-04 15:39:00', '2151-08-04 15:38:00', '2151-07-29 15:02:00', '2151-08-02 14:32:00', '2151-07-16 10:10:00', '2151-07-27 09:21:00', '2151-07-17 06:15:00', '2151-07-18 16:50:00', '2151-07-17 12:03:00', '2151-07-16 19:11:00', '2151-07-22 09:54:00', '2151-07-23 13:43:00', '2151-07-16 11:24:00', '2151-07-18 14:01:00', '2151-07-18 18:53:00', '2151-07-19 04:53:00', '2151-07-19 16:32:00', '2151-07-17 17:22:00', '2151-07-18 05:12:00', '2151-07-16 15:27:00', '2151-07-17 02:05:00', '2151-07-17 05:36:00', '2151-07-17 05:47:00'], 'admit_time': '2151-07-16 14:29:00', 'disch_time': '2151-08-04 19:10:00', 'admission_type': 'EMERGENCY', 'admission_location': 'EMERGENCY ROOM ADMIT', 'discharge_location': 'REHAB/DISTINCT PART HOSP', 'edreg_time': '2151-07-16 04:49:00', 'edout_time': 'UNKNOWN/NOT SPECIFIED'}], 'death_time': nan, '

What type of data is needed for the hospital:
1. Number of patients admitted in a month
2. Average duration of the patients
3. Admission location
4. Discharge location
5. Insurance
6. Language
7. Religion
8. Marital Status
9. Ethnicity
10. Emergency registered time
11. Emergency discharged time
12. Has chart events data

SUBJECT_ID{
    LANGUAGE,
    RELIGION,
    ETHNICITY,
    INSURANCE,
    MARITAL_STATUS,
    death_time,
    HADM_ID{
              ADMITTIME,
              DISCHTIME,
              DEATHTIME,
              EDREGTIME,
              EDOUTTIME,
              CHART_TIME[],
              STORE_TIME,
              DISCHARGE_LOCATION,
              ADMISSION_TYPE,
              ADMISSION_LOCATION,
              CATEGORY,
              CG_ID,
              ISERROR,
          }
    }
}

In [9]:
insuranceSet = {}
for i in range(46139):
    try:
        insuranceSet[(final_structure[i]['insurance'])]+=1
    except:
        insuranceSet[(final_structure[i]['insurance'])] = 1

In [10]:
print(insuranceSet)

{'Medicare': 20365, 'Private': 19289, 'Government': 1528, 'Medicaid': 4380, 'Self Pay': 577}


In [11]:
import pickle
with open('mimicData.pickle', 'wb') as handle:
    pickle.dump(final_structure, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
deathSet = set()
for i in range(46139):
    deathSet.add(final_structure[i]['death_time'])

In [36]:
import numpy as np
death_type_set = {}
count1 = 0
count2 = 0
for i in range(46139):
    try:
        if np.isnan(final_structure[i]['death_time']):
            count1+=1
            try:
                death_type_set[final_structure[i]['hospital'][0]['admission_type']]+=1
            except:
                death_type_set[final_structure[i]['hospital'][0]['admission_type']]=1
    except:
        count2+=1

In [40]:
print(death_type_set)
print(count2)

{'EMERGENCY': 27267, 'ELECTIVE': 6068, 'URGENT': 1002, 'NEWBORN': 7547}
4255


In [2]:
import pickle
objects = []
with (open("mimicData.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [5]:
for items in objects:
    print(items[0])
    break

{'pat_id': 22532, 'hospital': [{'stay_id': 167853.0, 'diagnosis': 'RT LOWER LOBE PNEUMONIA', 'chart_time': [nan, nan, nan, nan, nan, nan, nan, nan, '2151-07-16 05:01:00', '2151-08-04 15:39:00', '2151-08-04 15:38:00', '2151-07-29 15:02:00', '2151-08-02 14:32:00', '2151-07-16 10:10:00', '2151-07-27 09:21:00', '2151-07-17 06:15:00', '2151-07-18 16:50:00', '2151-07-17 12:03:00', '2151-07-16 19:11:00', '2151-07-22 09:54:00', '2151-07-23 13:43:00', '2151-07-16 11:24:00', '2151-07-18 14:01:00', '2151-07-18 18:53:00', '2151-07-19 04:53:00', '2151-07-19 16:32:00', '2151-07-17 17:22:00', '2151-07-18 05:12:00', '2151-07-16 15:27:00', '2151-07-17 02:05:00', '2151-07-17 05:36:00', '2151-07-17 05:47:00'], 'admit_time': '2151-07-16 14:29:00', 'disch_time': '2151-08-04 19:10:00', 'admission_type': 'EMERGENCY', 'admission_location': 'EMERGENCY ROOM ADMIT', 'discharge_location': 'REHAB/DISTINCT PART HOSP', 'edreg_time': '2151-07-16 04:49:00', 'edout_time': 'UNKNOWN/NOT SPECIFIED'}], 'death_time': nan, '